#                                 Identificación de Rostros

Para este proyecto, utilizaremos OpenCV para la manipulación de archivos y face_recognition (que usa modelos de Deep Learning de dlib) porque es significativamente más preciso que los métodos de OpenCV.

Preparación del Entorno
Si estás en Google Colab, primero debemos instalar las librerías necesarias. Ejecuta esta celda al inicio:

In [ ]:
# Instala bibliotecas
!pip install face_recognition opencv-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566166 sha256=a8b1e3edb34bb6d9452859d68f48cbb3e649fdadad4a4efb7de1be5ead37971d
  Stored in directory: /root/.cache/pip/wheels/8f/47/c8/f44c5aebb7507f7c8a2c0bd23151d732d0f0bd6884ad4ac635
Successfully built face-recognition-models


In [ ]:
# Estabecemos el ambiente operativo
import cv2
import face_recognition
import os
import datetime
import pandas as pd # Para el reporte
from google.colab.patches import cv2_imshow
#
# Aplicacion para identificar rostros, a partir de una imagen, buscara en un folder de imagenes o en un video
#   si existe la imagen buscada, generando una carpeta de resultados con las imagenes que empataron ademas de
#   un reporte.
#   Parametros de llamada:
#   opcion = folder o video (las imagenes se buscaran en una carpeta o en un video)
#   target_path = dirección de la imagen
#   search_path = dirección de la carpeta de imagenes a buscar si la opción es folder o dirección del video
#   output_dir = carpeta de resultados, si omitida se asume "resultados"
#
def aplicacion_facial_pro(opcion, target_path, search_path, output_dir="resultados"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    #  Prepara la referencia de comparación
    img_ref = face_recognition.load_image_file(target_path)
    encoding_ref = face_recognition.face_encodings(img_ref)

    if not encoding_ref:
        print("❌ Error: Rostro de referencia no detectado.")
        return

    target_encode = encoding_ref[0]
    registro_hallazgos = [] # Aquí guardaremos la data para el CSV

    print(f"🔍 Iniciando búsqueda en modo: {opcion}...")

    # --- MODO FOLDER ---
    # Basicamente lo que haremos sera: !) Seleccionar archivos de imagenes; 2) codificar las imagenes;
    # 3) comparar contra la imagen target, si verdadero; 4) dibujar un marco verde alrededor de la cara
    # de la imagen encontrada y guardarlas en el archivo de resultados, así como registrar los detalles
    # en la línea del reporte
    if opcion == 'folder':
        for archivo in os.listdir(search_path):
            if archivo.lower().endswith(('.jpg', '.png', '.jpeg')):
                ruta = os.path.join(search_path, archivo)
                img = cv2.imread(ruta)
                rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

                locs = face_recognition.face_locations(rgb)
                encodes = face_recognition.face_encodings(rgb, locs)

                for (top, right, bottom, left), encode in zip(locs, encodes):
                    if face_recognition.compare_faces([target_encode], encode, tolerance=0.6)[0]:
                        # Dibujar y guardar
                        cv2.rectangle(img, (left, top), (right, bottom), (0, 255, 0), 2)
                        cv2.imwrite(os.path.join(output_dir, f"match_{archivo}"), img)

                        # Registrar datos
                        registro_hallazgos.append({
                            'Origen': archivo,
                            'Tipo': 'Imagen',
                            'Tiempo/Detalle': 'N/A',
                            'Fecha_Proceso': datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        })
                        print(f"⭐ Encontrado en imagen: {archivo}")

    # --- MODO VIDEO ---
    # Basicamente lo que haremos sera: !) inicializar, entre ellos el contador de frames, frame_num = 0
    # pues para optimizar solo vamos a checar la imagen cada 15 frames;    2) codificar las imagenes;
    # 3) comparar contra la imagen target, si verdadero; 4) dibujar un marco verde alrededor de la cara
    # de la imagen encontrada y guardarlas en el archivo de resultados, así como registrar los detalles
    # en la línea del reporte
    elif opcion == 'video':
        cap = cv2.VideoCapture(search_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        frame_num = 0

        while cap.isOpened():
            ret, frame = cap.read()
            if not ret: break

            if frame_num % 15 == 0:
                rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                locs = face_recognition.face_locations(rgb)
                encodes = face_recognition.face_encodings(rgb, locs)

                for (top, right, bottom, left), encode in zip(locs, encodes):
                    if face_recognition.compare_faces([target_encode], encode)[0]:
                        segundos = frame_num / fps
                        tiempo = str(datetime.timedelta(seconds=segundos)).split('.')[0]

                        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                        nombre_frame = f"video_frame_{tiempo.replace(':','-')}.jpg"
                        cv2.imwrite(os.path.join(output_dir, nombre_frame), frame)

                        # Registrar datos
                        registro_hallazgos.append({
                            'Origen': os.path.basename(search_path),
                            'Tipo': 'Video',
                            'Tiempo/Detalle': tiempo,
                            'Fecha_Proceso': datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                        })
                        print(f"⭐ Encontrado en video al: {tiempo}")

            frame_num += 1
        cap.release()

    # --- GENERAR REPORTE ---
    if registro_hallazgos:
        df = pd.DataFrame(registro_hallazgos)
        csv_path = os.path.join(output_dir, "reporte_hallazgos.csv")
        df.to_csv(csv_path, index=False, encoding='utf-8-sig')
        print(f"\n✅ ¡Reporte generado con éxito! Archivo: {csv_path}")
    else:
        print("\n⚠️ No se encontraron coincidencias para el reporte.")

# --- LLAMADA ---
# aplicacion_facial_pro('video', 'objetivo.jpg', 'evidencia.mp4')

In [ ]:
#aplicacion_facial_pro(opcion='folder', target_path='/content/IMG_0347.JPG', search_path='/content/imagenes')